In [1]:
#working on writing useful functions to translate/examine HEK data output

import numpy as np
from shapely.geometry import Polygon
from shapely import wkt
from sunpy.time import parse_time

In [2]:
#HEK exports certain data as unicode strings. Convert these strings to relevant, useful datatypes

def unicode2datetime(time_array):
    for i, elem in time_array:
        time_array[i] = parse_time(elem)
        
def unicode2polygon(bbox_array):
    for i, elem in bbox_array:
        bbox_array[i] = wkt.loads(elem)

In [3]:
#function to determine whether 2 polygons overlap and, if so, in what manner they overlap.
#returns a list of booleans indicating different kinds of overlap 
#input parameters must both be shapely Polygons

def overlap(polygon1, polygon2):
    is_intersect = polygon1.intersects(polygon2)
    if is_intersect == False:
        is_crosses = False
        is_touches = False
        is_contained = False
        is_within = False
    else:
        is_crosses = polygon1.crosses(polygon2)
        is_touches = polygon1.touches(polygon2)
        is_contained = polygon1.contained(polygon2)
        is_within = polygon1.within(polygon2)
    return [is_intersect, is_crosses, is_touches, is_contained, is_within]

In [4]:
#function to determine the shortest distance between two preselected points on a sphere where we know 
#the radius of the sphere
#would be of use in calculating the distance between the centers of the flux emergence and flare
#nts: maybe change x,y inputs to be tuples later
def geodesic(x1, y1, x2, y2, r):
    chord = np.sqrt((x2-x1)**2 + (y2-y1)**2)
    s = r*np.arcsin(chord/(2*r))
    return s

In [5]:
#function to find shortest distance separating two areas
def short_dis_areas(polygon1_area, polygon2_area): #r? will have to add a column for flare area as that's not a given
#account for normalization by looking at disk center?


SyntaxError: invalid syntax (<ipython-input-5-4267f98e1ba1>, line 2)

In [ ]:
#function removes/sythesizes dupiclate entries for the same event